In [1]:
import pandas as pd
import math
import numpy as np
from collections import defaultdict
from math import radians, cos, sin, asin, sqrt
from pyecharts.charts import Scatter,Grid,Line
from pyecharts.commons.utils import JsCode
from pyecharts import options as opts

In [2]:
#基于嵌套字典
def Dijkstra(G,v0,INF=999):
    """ 
        使用 Dijkstra 算法计算指定点 v0 到图 G 中任意点的最短路径的距离        
        INF 为设定的无限远距离值        
        此方法不能解决负权值边的图    
    """
    book = set()
    minv = v0   # 源顶点到其余各顶点的初始路程
    dis = dict((k,INF) for k in G.keys())
    dis[v0] = 0 
    while len(book)<len(G):
        book.add(minv)  # 确定当期顶点的距离
        for w in G[minv]:   # 以当前点的中心向外扩散
            if(G[minv][w]>0):
                dmn=1-math.log(G[minv][w])
                if dis[minv] + dmn < dis[w]:# 如果从当前点扩展到某一点的距离小与已知最短距离      
                    dis[w] = dis[minv] + dmn         # 对已知距离进行更新
        new =INF    # 从剩下的未确定点中选择最小距离点作为新的扩散点
        minv=-1
        for v in dis.keys():
            if v in book: continue
            if dis[v] < new: 
                new = dis[v]
                minv = v
        if minv==-1:
            return dis
    return dis

#根据经纬度计算距离
def geodistance(lng1,lat1,lng2,lat2):
    lng1,lat1,lng2,lat2=map(radians,[float(lng1),float(lat1),float(lng2),float(lat2)])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2+cos(lat1)*cos(lat2)*sin(dlon/2)**2
    distance=2*asin(sqrt(a))*6371*1000
    distance=round(distance/1000,3)
    return distance

In [2]:
#画图函数
def scatter_visualmap_color(list2,data,corr_dis_time,province)->Scatter:
    c=(
        Scatter()
        .add_xaxis(list2)
        .add_yaxis("首例病例出现时间",data[2].values.tolist(),
                  label_opts=opts.LabelOpts(
                      is_show=False,
                        formatter=JsCode(
                            "function(params){return params.value[2];}" #通过定义JavaScript回调函数自定义标签
                        )
                    ),
        )
        .set_series_opts(effect_opts=opts.EffectOpts(symbol_size=5, color="yellow"),)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="首例病例出现时间与实际距离的关系",
                                      subtitle="相关系数="+str(corr_dis_time),
                                     pos_left='center'),
            legend_opts=opts.LegendOpts(pos_right=40,pos_top=15),
            xaxis_opts = opts.AxisOpts(
                          type_="value",#x轴数据类型是连续型的
                          min_=0,
                name='距'+province+'实际距离',
                name_location='middle'
                          ),
            yaxis_opts = opts.AxisOpts(
                          min_=5,
                name='出现时间/天',
                          )
        )
    )
    return c
def fitting_line(x,y) -> Line:
    line=(
        Line()
        .add_xaxis(x)
        .add_yaxis("拟合线",y,linestyle_opts=opts.LineStyleOpts(color="green", width=4, type_="dashed"),
                  label_opts=opts.LabelOpts(is_show=False))
    )
    return line

In [5]:
#读取数据，并存在嵌套字典实现的图中

graphs=pd.read_csv("data/流量数据/cityflux-fengchengqian122.txt",sep="\s+",header=None)
cityID=pd.read_csv('data/流量数据/cityID.txt',sep='\s+',header=None,encoding='GB2312')

d = defaultdict(defaultdict)
dmn = defaultdict(defaultdict)
for i in range(0,len(cityID)):  #插入对角线元素
    d[i][i]=0

for i in range(0,len(graphs)):
    d[graphs.iloc[i][0]][graphs.iloc[i][1]]=graphs.iloc[i][2]
Dijkstra(d,v0=1)

{0: 11.361157912887862,
 1: 0,
 2: 5.846223157845869,
 3: 10.325739967991161,
 4: 12.555034353596024,
 5: 8.912739768937833,
 6: 3.253414048987596,
 7: 11.991011315318396,
 8: 6.9646278931544225,
 9: 8.228895720117096,
 10: 6.799852943406934,
 11: 9.838947196616587,
 12: 6.954168762364285,
 13: 6.823918275896181,
 14: 5.019107228037821,
 15: 6.906388374752862,
 16: 6.794911408067303,
 17: 11.22164831646985,
 18: 9.356593580026539,
 19: 9.462004855817367,
 20: 11.44236044076257,
 21: 5.286571010901589,
 22: 2.232283119585128,
 23: 6.920074551021309,
 24: 10.634239166893222,
 25: 6.764807176493975,
 26: 4.149629652131386,
 27: 6.730948142728971,
 28: 9.998346155370864,
 29: 9.595978561847312,
 30: 6.799852943406934,
 31: 4.2100647565896505,
 32: 10.099949731234958,
 33: 9.216970682414086,
 34: 11.388700274495537,
 35: 7.126597221099479,
 36: 4.206058321157586,
 37: 10.435190961783551,
 38: 9.519461910842155,
 39: 9.851437261810343,
 40: 5.727450820696924,
 41: 10.96244223926,
 42: 5.8895

In [4]:
# 计算每对顶点之间的有效距离
for i in range(0,len(cityID)):
    dmn[i]=Dijkstra(d,v0=i)
#输出为文件
dmn=pd.DataFrame(dmn)
dmn.to_csv('data/流量数据/city_dmn_after.txt',sep='\t',index=False,header=None)

In [12]:
# 计算每对城市顶点之间的地理球面距离,并输出为文件
cityInf=pd.read_csv('data/流量数据/city informationEng.txt',header=None,sep='\s+',encoding='GB2312')

gmn=[[0 for i in range(0,333)] for i in range(0,333)] #实际地理距离初始化

gmn=[[geodistance(float(cityInf.iloc[i][4]),float(cityInf.iloc[i][5]),float(cityInf.iloc[j][4]),float(cityInf.iloc[j][5])) for j in range(1,333)]for i in range(1,333)]

gmn=pd.DataFrame(gmn)
gmn.head()
gmn.to_csv('data/流量数据/city_gmn.txt',sep='\t',index=False,header=None)

In [3]:
#各城市首例病例出现时间，武汉市为第零天。有空缺值，因此要剔除这些行
#武汉到部分城市的有效距离为无穷大，也要剔除这些城市
first_day=pd.read_csv('data/流量数据/city.txt',sep='\s+',header=None,encoding='GB2312')
first_day = first_day.fillna(-1)   #填补空缺值
bool_row1=[True if i>0 else False for i in first_day[2]]   #用于剔除不合格日期的行
dmn=pd.read_csv('data/流量数据/city_dmn.txt',sep='\s+',header=None,encoding='GB2312')
gmn=pd.read_csv('data/流量数据/city_gmn.txt',sep='\s+',header=None,encoding='GB2312')
gmn_wh=pd.Series(gmn.iloc[bool_row1,71])
time_g=pd.Series(first_day.iloc[bool_row1,2])
first_day_g=first_day.iloc[bool_row1,:]
bool_row2=[False if i==999 else True for i in dmn[71]]    #用于剔除不合格有效距离的行
bool_row=[i if j==True else False for i,j in zip(bool_row1,bool_row2)]  #将两者结合，同时提出time,dmn
dmn_wh=pd.Series(dmn.iloc[bool_row,71])
time_d=pd.Series(first_day.iloc[bool_row,2])  #首例病例出现时间
first_day_d=first_day.iloc[bool_row,:]

In [4]:
# 拟合
d_k,d_b= np.polyfit(dmn_wh, time_d, 1)
g_k,g_b=np.polyfit(gmn_wh,time_g,1)

#计算以有效距离,实际距离为回归变量的相对残差 
R_d=0
for i,j in zip(dmn_wh,time_d):
    R_d+=((j-i*d_k-d_b)/j)**2
R_g=0
for i,j in zip(gmn_wh,time_g):
    R_g+=((j-i*g_k-g_b)/j)**2
R_d, R_g
#(10.900321006589893, 13.647033040457401)
# 0.4051629709755104, 10.352478171354294, 0.0015295767092739058 13.174604809501481

(10.900321006589893, 13.647033040457401)

In [138]:
# 计算有效距离与首例病例出现时间的相关性    
corr_dis_time=round(dmn_wh.corr(time_d),5)
corr_dis_time
z1 = np.polyfit(dmn_wh, time_d, 1)  #一次多项式拟合，相当于线性拟合
line=fitting_line([0,15],[z1[1],z1[0]*15+z1[1]])
# 生成图片
graph_wh=scatter_visualmap_color(dmn_wh,first_day_d,corr_dis_time,'Wuhan')
graph_wh.render(path='test_wh_d.html')
graph_wh.overlap(line)
graph_wh.render(path='test_wh_d_fitting.html')

'I:\\biyesheji\\test_wh_d_fitting.html'

In [5]:
# 计算实际距离与首例病例出现时间的相关性 
corr_g_time_wh=round(gmn_wh.corr(time_g),5)
z1 = np.polyfit(gmn_wh, time_g, 1)  #一次多项式拟合，相当于线性拟合
line=fitting_line([0,3500],[z1[1],z1[0]*3500+z1[1]])
# 生成图片
graph_wh=scatter_visualmap_color(gmn_wh,first_day_g,corr_g_time_wh,'Wuhan')
graph_wh.render(path='test_wh_g.html')
graph_wh.overlap(line)
graph_wh.render(path='test_wh_g_fitting.html')

'I:\\biyesheji\\test_wh_g_fitting.html'